<a href="https://colab.research.google.com/github/alice202122/2competitionDL/blob/main/Self_supervised.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Set up the environment

In [4]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [5]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D, Dropout
from keras.callbacks import LearningRateScheduler
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import SGD
from IPython.display import Image, display
from tensorflow.keras.optimizers import Adam
from keras.regularizers import l2
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import matplotlib.cm as cm
import cv2 #image operations

In [6]:
import pickle
import os
import random
import pandas as pd
import seaborn as sns
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

from PIL import Image

from tensorflow.keras import layers, models, backend, applications, regularizers, initializers
from tensorflow.keras.layers import Layer
from tqdm import tqdm
from tensorflow.python.ops import math_ops

from tensorflow.keras import layers
from tensorflow.keras.callbacks import TensorBoard
from sklearn.utils import class_weight
from keras import layers, Sequential
from keras.layers import Input, Add, Dense, Activation, ZeroPadding1D, Flatten, GlobalAveragePooling1D,  Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dropout,concatenate, BatchNormalization, Reshape
from keras.models import Model
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import LearningRateScheduler
from tensorflow.keras import regularizers


tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)

2.8.2


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
# Random seed for reproducibility
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [9]:
%cd /content/drive/MyDrive/Tesi

/content/drive/MyDrive/Tesi


In [ ]:
!unzip DataSemiSupervised.zip

Archive:  DataSemiSupervised.zip
   creating: DataSemiSupervised/Labeled/
   creating: DataSemiSupervised/Labeled/Healthy/
  inflating: DataSemiSupervised/Labeled/Healthy/1040_Mazzarelli_Domenico Marino__185_Angio Retina_OD_2016-04-14_16-52-52_M_1944-09-18_Enface-304x304-Superficial.png  
  inflating: DataSemiSupervised/Labeled/Healthy/1040_Mazzarelli_Domenico Marino__185_Angio Retina_OS_2016-04-14_16-56-03_M_1944-09-18_Enface-304x304-Superficial.png  
  inflating: DataSemiSupervised/Labeled/Healthy/1095_prospero_giovanni__1671_Angio Retina_OD_2022-01-27_17-25-08_M_1942-12-14_Enface-304x304-Superficial.png  
  inflating: DataSemiSupervised/Labeled/Healthy/1095_prospero_giovanni__66_Angio Retina_OS_2016-02-18_16-21-23_M_1942-12-14_Enface-304x304-Superficial.png  
  inflating: DataSemiSupervised/Labeled/Healthy/1118_Nitti_Anna Maria__308_Angio Retina_OD_2016-06-13_17-01-33_F_1944-10-01_Enface-304x304-Superficial.png  
  inflating: DataSemiSupervised/Labeled/Healthy/1118_Nitti_Anna Maria_

#Loading the data

In [7]:
!pip install tensorflow-datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
AUTOTUNE = tf.data.AUTOTUNE
shuffle_buffer = 99
# The below two values are taken from https://www.tensorflow.org/datasets/catalog/stl10
labeled_train_images = 99
unlabeled_images =599

temperature = 0.1
queue_size = 599
contrastive_augmenter = {
    "brightness": 0.5,
    "name": "contrastive_augmenter",
    "scale": (0.2, 1.0),
}
classification_augmenter = {
    "brightness": 0.2,
    "name": "classification_augmenter",
    "scale": (0.5, 1.0),
}
input_shape = (304, 304, 3)
width = 128
num_epochs = 25
steps_per_epoch = 5

In [11]:
# Dataset folders 
dataset_dir = "/content/drive/MyDrive/Tesi/DataSemiSupervised" #path to the folder
train_unlabeled = os.path.join(dataset_dir, 'train_unlabeled') #we are putting together these string
test_labeled = os.path.join(dataset_dir, 'test_labeled')
labeled = os.path.join(dataset_dir,'Labeled')
CATEGORIES=['Healthy', 'Unhealthy']

In [12]:
unlabeled_batch_size = unlabeled_images // steps_per_epoch
labeled_batch_size = labeled_train_images // steps_per_epoch
batch_size = unlabeled_batch_size + labeled_batch_size

In [17]:
im_gen = ImageDataGenerator()
unlabeled_train_data= tf.keras.utils.image_dataset_from_directory(train_unlabeled,
                                                 labels=None,
                                                 batch_size=unlabeled_batch_size, 
                                                 image_size=(304, 304),
                                                 shuffle=True)
labeled_train_data=im_gen.flow_from_directory(labeled,
                                                 class_mode='binary',
                                                 batch_size=labeled_batch_size,
                                                 classes= ('Healthy', 'Unhealthy'),
                                                 target_size=(304, 304), 
                                                 shuffle=True)
labeled_test_data=im_gen.flow_from_directory(test_labeled,
                                                 class_mode='binary',
                                                 batch_size=batch_size,
                                                 classes= ('Healthy', 'Unhealthy'),
                                                 target_size=(304, 304), 
                                                 shuffle=False)

Found 599 files belonging to 1 classes.
Found 99 images belonging to 2 classes.
Found 128 images belonging to 2 classes.


## Data preparation

In [33]:
labeled_train_data= tf.data.Dataset.from_generator(
    lambda: labeled_train_data ,  # Our generator 
    output_types = (tf.float32 , tf.float32) , # How we're expecting our output dtype
    output_shapes = ([labeled_batch_size , 304 , 304 , 3] , [labeled_batch_size , ]) # How we're expecting our output shape
)
labeled_test_data= tf.data.Dataset.from_generator(
    lambda: labeled_test_data ,  # Our generator 
    output_types = (tf.float32 , tf.float32) , # How we're expecting our output dtype
    output_shapes = ([batch_size , 304 , 304 , 3] , [batch_size , ]) # How we're expecting our output shape
)
unlabeled_train_data= tf.data.Dataset.from_generator(
    lambda: unlabeled_train_data ,  # Our generator 
    output_types = (tf.float32) , # How we're expecting our output dtype
    output_shapes = ([unlabeled_batch_size , 304 , 304 , 3] )) # How we're expecting our output 
labeled_train_data, labeled_test_data, unlabeled_train_data

(<FlatMapDataset element_spec=(TensorSpec(shape=(19, 304, 304, 3), dtype=tf.float32, name=None), TensorSpec(shape=(19,), dtype=tf.float32, name=None))>,
 <FlatMapDataset element_spec=(TensorSpec(shape=(138, 304, 304, 3), dtype=tf.float32, name=None), TensorSpec(shape=(138,), dtype=tf.float32, name=None))>,
 <FlatMapDataset element_spec=TensorSpec(shape=(119, 304, 304, 3), dtype=tf.float32, name=None)>)

In [15]:
def normalize_image(image, label):
    return tf.cast(image, tf.float32) / 255., label
  

In [16]:
def normalize_unl_image(image):
    return image/255.
  


In [30]:
from sklearn.metrics.pairwise import normalize
# Map preprocessing function to training data (and paralellize)
unlabeled_train_data = unlabeled_train_data.map(normalize_unl_image, num_parallel_calls=tf.data.AUTOTUNE)
# Shuffle train_data and enable prefetch (load it faster)
unlabeled_train_data = unlabeled_train_data.shuffle(buffer_size=10 * unlabeled_batch_size).batch(unlabeled_batch_size)

labeled_train_data= labeled_train_data.map(normalize_image, num_parallel_calls=tf.data.AUTOTUNE)
labeled_train_data = labeled_train_data.shuffle(buffer_size=10 * labeled_batch_size).batch(labeled_batch_size)

labeled_test_data= labeled_test_data.map(normalize_image, num_parallel_calls=tf.data.AUTOTUNE)
labeled_test_data = labeled_test_data.batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)

In [20]:
"""unlabeled_train_data= unlabeled_train_data.apply(tf.data.experimental.assert_cardinality(599)) #number of samples
labeled_train_data= labeled_train_data.apply(tf.data.experimental.assert_cardinality(99))
labeled_test_data= labeled_test_data.apply(tf.data.experimental.assert_cardinality(128))"""



In [38]:
dataset= zip(unlabeled_train_data, labeled_train_data)

In [31]:
train_dataset = tf.data.Dataset.zip(
        (unlabeled_train_data, labeled_train_data)
    ).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
list(train_dataset.as_numpy_iterator())

In [32]:
len(train_dataset)

6

## Data augmentation

In [23]:
class RandomResizedCrop(layers.Layer):
    def __init__(self, scale, ratio):
        super(RandomResizedCrop, self).__init__()
        self.scale = scale
        self.log_ratio = (tf.math.log(ratio[0]), tf.math.log(ratio[1]))

    def call(self, images):
        batch_size = tf.shape(images)[0]
        height = tf.shape(images)[1]
        width = tf.shape(images)[2]

        random_scales = tf.random.uniform((batch_size,), self.scale[0], self.scale[1])
        random_ratios = tf.exp(
            tf.random.uniform((batch_size,), self.log_ratio[0], self.log_ratio[1])
        )

        new_heights = tf.clip_by_value(tf.sqrt(random_scales / random_ratios), 0, 1)
        new_widths = tf.clip_by_value(tf.sqrt(random_scales * random_ratios), 0, 1)
        height_offsets = tf.random.uniform((batch_size,), 0, 1 - new_heights)
        width_offsets = tf.random.uniform((batch_size,), 0, 1 - new_widths)

        bounding_boxes = tf.stack(
            [
                height_offsets,
                width_offsets,
                height_offsets + new_heights,
                width_offsets + new_widths,
            ],
            axis=1,
        )
        images = tf.image.crop_and_resize(
            images, bounding_boxes, tf.range(batch_size), (height, width)
        )
        return images

In [24]:
class RandomBrightness(layers.Layer):
    def __init__(self, brightness):
        super(RandomBrightness, self).__init__()
        self.brightness = brightness

    def blend(self, images_1, images_2, ratios):
        return tf.clip_by_value(ratios * images_1 + (1.0 - ratios) * images_2, 0, 1)

    def random_brightness(self, images):
        # random interpolation/extrapolation between the image and darkness
        return self.blend(
            images,
            0,
            tf.random.uniform(
                (tf.shape(images)[0], 1, 1, 1), 1 - self.brightness, 1 + self.brightness
            ),
        )

    def call(self, images):
        images = self.random_brightness(images)
        return images

In [25]:
def augmenter(brightness, name, scale):
    return keras.Sequential(
        [
            layers.Input(shape=input_shape),
            layers.Rescaling(1 / 255),
            layers.RandomFlip("horizontal"),
            RandomResizedCrop(scale=scale, ratio=(3 / 4, 4 / 3)),
            RandomBrightness(brightness=brightness),
        ],
        name=name,
    )

In [26]:
def encoder():
    return keras.Sequential(
        [
            layers.Input(shape=input_shape),
            layers.Conv2D(width, kernel_size=3, strides=2, activation="relu"),
            layers.Conv2D(width, kernel_size=3, strides=2, activation="relu"),
            layers.Conv2D(width, kernel_size=3, strides=2, activation="relu"),
            layers.Conv2D(width, kernel_size=3, strides=2, activation="relu"),
            layers.Flatten(),
            layers.Dense(width, activation="relu"),
        ],
        name="encoder",
    )


##NCLR 

In [27]:
class NNCLR(keras.Model):
    def __init__(
        self, temperature, queue_size,
    ):
        super(NNCLR, self).__init__()
        self.probe_accuracy = keras.metrics.SparseCategoricalAccuracy()
        self.correlation_accuracy = keras.metrics.SparseCategoricalAccuracy()
        self.contrastive_accuracy = keras.metrics.SparseCategoricalAccuracy()
        self.probe_loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

        self.contrastive_augmenter = augmenter(**contrastive_augmenter)
        self.classification_augmenter = augmenter(**classification_augmenter)
        self.encoder = encoder()
        self.projection_head = keras.Sequential(
            [
                layers.Input(shape=(width,)),
                layers.Dense(width, activation="relu"),
                layers.Dense(width),
            ],
            name="projection_head",
        )
        self.linear_probe = keras.Sequential(
            [layers.Input(shape=(width,)), layers.Dense(2)], name="linear_probe"
        )
        self.temperature = temperature

        feature_dimensions = self.encoder.output_shape[1]
        self.feature_queue = tf.Variable(
            tf.math.l2_normalize(
                tf.random.normal(shape=(queue_size, feature_dimensions)), axis=1
            ),
            trainable=False,
        )

    def compile(self, contrastive_optimizer, probe_optimizer, **kwargs):
        super(NNCLR, self).compile(**kwargs)
        self.contrastive_optimizer = contrastive_optimizer
        self.probe_optimizer = probe_optimizer

    def nearest_neighbour(self, projections):
        support_similarities = tf.matmul(
            projections, self.feature_queue, transpose_b=True
        )
        nn_projections = tf.gather(
            self.feature_queue, tf.argmax(support_similarities, axis=1), axis=0
        )
        return projections + tf.stop_gradient(nn_projections - projections)

    def update_contrastive_accuracy(self, features_1, features_2):
        features_1 = tf.math.l2_normalize(features_1, axis=1)
        features_2 = tf.math.l2_normalize(features_2, axis=1)
        similarities = tf.matmul(features_1, features_2, transpose_b=True)

        batch_size = tf.shape(features_1)[0]
        contrastive_labels = tf.range(batch_size)
        self.contrastive_accuracy.update_state(
            tf.concat([contrastive_labels, contrastive_labels], axis=0),
            tf.concat([similarities, tf.transpose(similarities)], axis=0),
        )

    def update_correlation_accuracy(self, features_1, features_2):
        features_1 = (
            features_1 - tf.reduce_mean(features_1, axis=0)
        ) / tf.math.reduce_std(features_1, axis=0)
        features_2 = (
            features_2 - tf.reduce_mean(features_2, axis=0)
        ) / tf.math.reduce_std(features_2, axis=0)

        batch_size = tf.shape(features_1, out_type=tf.float32)[0]
        cross_correlation = (
            tf.matmul(features_1, features_2, transpose_a=True) / batch_size
        )

        feature_dim = tf.shape(features_1)[1]
        correlation_labels = tf.range(feature_dim)
        self.correlation_accuracy.update_state(
            tf.concat([correlation_labels, correlation_labels], axis=0),
            tf.concat([cross_correlation, tf.transpose(cross_correlation)], axis=0),
        )

    def contrastive_loss(self, projections_1, projections_2):
        projections_1 = tf.math.l2_normalize(projections_1, axis=1)
        projections_2 = tf.math.l2_normalize(projections_2, axis=1)

        similarities_1_2_1 = (
            tf.matmul(
                self.nearest_neighbour(projections_1), projections_2, transpose_b=True
            )
            / self.temperature
        )
        similarities_1_2_2 = (
            tf.matmul(
                projections_2, self.nearest_neighbour(projections_1), transpose_b=True
            )
            / self.temperature
        )

        similarities_2_1_1 = (
            tf.matmul(
                self.nearest_neighbour(projections_2), projections_1, transpose_b=True
            )
            / self.temperature
        )
        similarities_2_1_2 = (
            tf.matmul(
                projections_1, self.nearest_neighbour(projections_2), transpose_b=True
            )
            / self.temperature
        )

        batch_size = tf.shape(projections_1)[0]
        contrastive_labels = tf.range(batch_size)
        loss = keras.losses.sparse_categorical_crossentropy(
            tf.concat(
                [
                    contrastive_labels,
                    contrastive_labels,
                    contrastive_labels,
                    contrastive_labels,
                ],
                axis=0,
            ),
            tf.concat(
                [
                    similarities_1_2_1,
                    similarities_1_2_2,
                    similarities_2_1_1,
                    similarities_2_1_2,
                ],
                axis=0,
            ),
            from_logits=True,
        )

        self.feature_queue.assign(
            tf.concat([projections_1, self.feature_queue[:-batch_size]], axis=0)
        )
        return loss

    def train_step(self, data):
        (unlabeled_images, _), (labeled_images, labels) = data
        images = tf.concat((unlabeled_images, labeled_images), axis=0)
        augmented_images_1 = self.contrastive_augmenter(images)
        augmented_images_2 = self.contrastive_augmenter(images)

        with tf.GradientTape() as tape:
            features_1 = self.encoder(augmented_images_1)
            features_2 = self.encoder(augmented_images_2)
            projections_1 = self.projection_head(features_1)
            projections_2 = self.projection_head(features_2)
            contrastive_loss = self.contrastive_loss(projections_1, projections_2)
        gradients = tape.gradient(
            contrastive_loss,
            self.encoder.trainable_weights + self.projection_head.trainable_weights,
        )
        self.contrastive_optimizer.apply_gradients(
            zip(
                gradients,
                self.encoder.trainable_weights + self.projection_head.trainable_weights,
            )
        )
        self.update_contrastive_accuracy(features_1, features_2)
        self.update_correlation_accuracy(features_1, features_2)
        preprocessed_images = self.classification_augmenter(labeled_images)

        with tf.GradientTape() as tape:
            features = self.encoder(preprocessed_images)
            class_logits = self.linear_probe(features)
            probe_loss = self.probe_loss(labels, class_logits)
        gradients = tape.gradient(probe_loss, self.linear_probe.trainable_weights)
        self.probe_optimizer.apply_gradients(
            zip(gradients, self.linear_probe.trainable_weights)
        )
        self.probe_accuracy.update_state(labels, class_logits)

        return {
            "c_loss": contrastive_loss,
            "c_acc": self.contrastive_accuracy.result(),
            "r_acc": self.correlation_accuracy.result(),
            "p_loss": probe_loss,
            "p_acc": self.probe_accuracy.result(),
        }

    def test_step(self, data):
        labeled_images, labels = data

        preprocessed_images = self.classification_augmenter(
            labeled_images, training=False
        )
        features = self.encoder(preprocessed_images, training=False)
        class_logits = self.linear_probe(features, training=False)
        probe_loss = self.probe_loss(labels, class_logits)

        self.probe_accuracy.update_state(labels, class_logits)
        return {"p_loss": probe_loss, "p_acc": self.probe_accuracy.result()}

In [29]:
model = NNCLR(temperature=temperature, queue_size=queue_size)
model.compile(
    contrastive_optimizer=keras.optimizers.Adam(),
    probe_optimizer=keras.optimizers.Adam(),
)
pretrain_history = model.fit(
    train_dataset, epochs=num_epochs, validation_data=labeled_test_data
)

Epoch 1/25


OperatorNotAllowedInGraphError: ignored